Testing the code from Kjersti`s github calculating the mixed layer depth. 

In [67]:
#CODE implemented from Kjersti:
#from roppy import SGrid
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import griddata
from glob import glob
import time
import xarray as xr
import sys
import os
from dens_func import dens

In [80]:
def MLD(pot_dens, z):
    '''
    Calculate Mixed Layer Depth (MLD) based on potential density profile and depth.
    MLD is where the potential density exceeds a threshold, here set to be surface
    potential density + 0.03 kgm-3.

    Parameters:
    - pot_dens: 1D numpy array of potential density [kg/m^3]
    - z: 1D numpy array of corresponding depth levels [m] (negative downward)

    Returns:
    - mld: scalar value of MLD [m], or local water depth if no depth exceeds threshold,
    meaning full water column is mixed.
    '''
    # Remove NaNs
    valid = ~np.isnan(pot_dens)
    pot_dens = pot_dens[valid]
    z = z[valid]

    if len(pot_dens) == 0:
        return np.nan

    # Surface density
    surface_density = pot_dens[0]
    threshold = surface_density + 0.03  # MLD is where density exceeds surface + 0.03

    # Find where density exceeds threshold
    exceed = np.where(pot_dens >= threshold)[0]

    if exceed.size == 0:
        return print(f"No depth exceeds treshold. First value of z is returned: {z[-1]}")  # no depth exceeds threshold

    # Return the first depth where threshold is exceeded
    return print(f"The first depth where the treshold is exceeded is: {z[exceed[0]]}") 


In [69]:
ds_2024 = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240330.nc').isel(s_rho = -1, time = 0, X = 360, Y = 760)

In [70]:
ds_2024.head()

<xarray.Dataset> Size: 236B
Dimensions:           (s_w: 5)
Coordinates:
    time              datetime64[ns] 8B 2024-03-30
    s_rho             float64 8B -0.004904
    X                 float64 8B 2.88e+05
    Y                 float64 8B 6.08e+05
  * s_w               (s_w) float64 40B -1.0 -0.96 -0.9208 -0.8823 -0.8446
    lon               float64 8B ...
    lat               float64 8B ...
Data variables: (12/18)
    Uwind_eastward    float32 4B ...
    Vwind_northward   float32 4B ...
    zeta              float32 4B ...
    ubar_eastward     float32 4B ...
    vbar_northward    float32 4B ...
    u_eastward        float32 4B ...
    ...                ...
    hc                float64 8B ...
    Cs_r              float64 8B ...
    Cs_w              (s_w) float64 40B ...
    h                 float32 4B ...
    projection_stere  int32 4B ...
    depth             int32 4B ...
Attributes: (12/33)
    id:                      15f95603-12d1-4e0f-8cbe-33946594447f
    naming_authority:        no.met
    operational_status:      scientific
    iso_topic_category:      oceans
    activity_type:           Numerical Simulation
    keywords_vocabulary:     GCMDSK:GCMD Science Keywords:https://gcmd.earthd...
    ...                      ...
    project:                 Norkyst_v3
    license:                 https://spdx.org/licenses/CC-BY-4.0 (CC-BY-4.0)
    title:                   Norkyst-800m - ROMS, Norkyst-800m ocean hindcast...
    summary:                 Norkyst-800m (Norwegian Coast 800m horizontal re...
    title_no:                Hindcast prognoser fra havmodellen Norkyst-800m,...
    summary_no:              NorKyst-800m (Norske kystområder med 800m horiso...

In [71]:
temp = ds_2024.temperature.transpose() 
salinity = ds_2024.salinity.transpose() 


In [72]:
#Note: Long name of temp is pot temp, meaning the pressure effects are already counted for and does not need to be changed for the NorKyst model. 
#If applied to the infer results - double check wether the temperature is potential temp or not when comparing mld for more layers than the surface.
pot_dens = dens(temp, salinity)
pot_dens = pot_dens.values

In [73]:
#Check if the shape of pot_dens matches the needed shape to use in the mld function
"""
print(f"The shape of the potential density is: {pot_dens.shape}")
print(f"The dims of potential density is: {pot_dens.dims}")
pot_dens = pot_dens.values
print(f"The shape of the potential density is: {pot_dens.shape}")
#pot_dens = pot_dens.stack()
"""

'\nprint(f"The shape of the potential density is: {pot_dens.shape}")\nprint(f"The dims of potential density is: {pot_dens.dims}")\npot_dens = pot_dens.values\nprint(f"The shape of the potential density is: {pot_dens.shape}")\n#pot_dens = pot_dens.stack()\n'

In [74]:
def finding_2m_layer(ds_name):
    #Define necessary variables used for the transformation from s_layer to depth
    hc = ds_name["hc"] #Critical depth for stretching
    cs_r = ds_name["Cs_r"] #stretching curve at rho points
    zeta = ds_name["zeta"] #.fillna(0) #free-surface 
    H = ds_name["h"] #bathymetry at rho-points (depth)
    #Vtransform = ds_name["Vtransform"] Not in this dataset
    s_rho = ds_name["s_rho"] #range 1,40. 40 is surface layer

    #Transformation process
    #if Vtransform == 1:
        #Z_0_rho = hc * (s_rho - cs_r) + cs_r * H
        #z_rho = Z_0_rho + zeta * (1+Z_0_rho/H)
    #elif Vtransform == 2:
    Z_0_rho = (hc * s_rho + cs_r * H) / (hc + H)
    z_rho = zeta + (zeta + H) * Z_0_rho

    ds_name.coords["z_rho"] = z_rho.transpose() #Corrects the dimensions

In [75]:
finding_2m_layer(ds_2024) #z_rho explains the depth for a given grid cell, not sure if this is ideal yet. We will find out. 

In [76]:
#Check if it works - and check shape
#ds_2024.z_rho
z_rho = ds_2024.z_rho.values
print(f"The shape of z_rho is: {z_rho}")

The shape of z_rho is: -0.3047012858795581


In [81]:
MLD(pot_dens = pot_dens, z = z_rho)

No depth exceeds treshold. First value of z is returned: -0.3047012858795581


In [85]:
#Testing another area
ds_2024_new = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240330.nc').isel(s_rho = -1, time = 0, X = 413, Y = 910)
